In [1]:
import sys
import glob
import serial
import time
import visdom, gc
import numpy as np
import os
from matplotlib import pyplot as plt
from datetime import datetime, timezone

#sys.path.append(r"")
from liveplot.plotter import LivePlot

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
def save_data(t, y, filename):
    if isinstance(y, list):
        datapoint = np.array([[t] + y])
        data_format = '%s,'+'%s,'*(len(y)-1)+'%s'
    else:
        datapoint = np.array([[t, str(y)]])
        data_format = '%s,%s'
        
    if os.path.exists(filename):
        with open(filename, "ab") as f:
            np.savetxt(f, datapoint, delimiter=',', fmt=data_format)   
    else:
        np.savetxt(filename, datapoint, delimiter=',', fmt=data_format, 
                   header=r"", comments="")

In [4]:
def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result


if __name__ == '__main__':
    print(serial_ports())

['/dev/tty.Bluetooth-Incoming-Port', '/dev/tty.usbmodem1101']


Replace the first argument in the `serial.Serial` function to the USB port used by the Arduino.

In [5]:
ser = serial.Serial('/dev/tty.usbmodem1101', 9600)

In [6]:
co2_vals = list()
temp_vals = list()
hum_vals = list()

p = LivePlot()
# p.boot()
p.clear_windows()
t0 = time.time()
# today = time.strftime("%Y%m%d")

Setting up a new session...


Closing plot window CO2 concentration...


In [7]:
ser.flush()

Test that the serial connection is responding:

In [9]:
ser.readline()

b'Found SGP30 serial #024E78E9\r\n'

After executing the cell below, make sure to check the visdom server in your webbrowser, and make sure that the log files are being written to. This cell runs indefinitely and the notebook should be kept open to collect the data.

In [ ]:
plot_points = 300
log_interval = 20

# Savepath for the data
basepath = r"./data" 

t_log = time.time()
last_log_time = 0

pm1_avg, pm2p5_avg, pm10_avg, voc_avg = list(), list(), list(), list()
while True: 
    x = ser.readline()
    try:
        raw_values = x.decode().split(',')
        co2_str, temp_str, hum_str, pres_str, alt_str, voc_str, eco2_str, pm1_str, pm2p5_str, pm10_str = raw_values
        co2_ppm = int(co2_str.strip(r'\r\n').strip(r'OK '))
        temp_c = float(temp_str)
        hum_pct = float(hum_str)
        pres_hpa = float(pres_str)
        alt_m = float(alt_str)
        voc = float(voc_str)
        pm1 = float(pm1_str)
        pm2p5 = float(pm2p5_str)
        pm10 = float(pm10_str)
        
        if co2_ppm > 380: #anything below this value can't be physical
            co2_vals.append(co2_ppm)
            temp_vals.append(temp_c)
            hum_vals.append(hum_pct)
            t = time.time() - t0
            # 300 pts is appr. 10 minutes.
            p.liveplot_1d(t, co2_ppm, window_name="CO2 concentration", max_pts=plot_points, xlabel="Time (s)", ylabel="CO2 (ppm)")

    
            pm1_avg.append(pm1)
            pm2p5_avg.append(pm2p5)
            pm10_avg.append(pm10)
            voc_avg.append(voc)
    
            # Only save the data when log_interval has elapsed.
            if time.time() - last_log_time > log_interval:
                timestamp = time.strftime("%d-%m-%y,%H:%M:%S")
                today = time.strftime("%y-%m-%d")
                

                save_data(timestamp, [temp_c, hum_pct, pres_hpa, alt_m], os.path.join(basepath, f"THP {today}.log"))
                save_data(timestamp, [co2_ppm, np.mean(pm1_avg), np.mean(pm2p5_avg), np.mean(pm10_avg), np.mean(voc_avg)], 
                          os.path.join(basepath, f"AQI {today}.log"))
                
                last_log_time = time.time()
                # Reset the averaging when a log as occurred.
                pm1_avg, pm2p5_avg, pm10_avg, voc_avg = list(), list(), list(), list()
            
            # Even if your log_interval is larger than 1 second, you must still sleep here 1 second 
            # otherwise the buffer fills up and the data from the arduino is no longer current.
            # The sleep time in the arduino is roughly 2 seconds for our sensor.
            time.sleep(1.0)
        else:
            time.sleep(0.25)
    except:
        pass